# Text Preprocessing Tutorial for AG News Classification

## Overview

This notebook demonstrates comprehensive text preprocessing techniques following methodologies from:
- Pennington et al. (2014): "GloVe: Global Vectors for Word Representation"
- Bojanowski et al. (2017): "Enriching Word Vectors with Subword Information"
- Kudo & Richardson (2018): "SentencePiece: A simple and language independent subword tokenizer"

### Tutorial Objectives
1. Implement text cleaning and normalization
2. Apply tokenization for transformer models
3. Prepare data compatible with model training
4. Extract features for analysis
5. Optimize preprocessing pipeline
6. Save preprocessed data for training

Author: Võ Hải Dũng  
Email: vohaidung.work@gmail.com  
Date: 2025

## 1. Environment Setup

In [ ]:
# Standard library imports
import sys
import re
import string
import unicodedata
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from collections import Counter
import warnings

# Data manipulation and NLP imports
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer
import nltk

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Project imports
PROJECT_ROOT = Path("../..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

from src.data.datasets.ag_news import AGNewsDataset, AGNewsConfig
from src.data.preprocessing.text_cleaner import TextCleaner, CleaningConfig
from src.data.preprocessing.tokenization import Tokenizer, TokenizationConfig
from src.data.preprocessing.feature_extraction import FeatureExtractor, FeatureExtractionConfig
from src.data.preprocessing.sliding_window import SlidingWindowProcessor
from src.data.preprocessing.prompt_formatter import PromptFormatter
from src.utils.io_utils import safe_load, safe_save, ensure_dir
from src.utils.logging_config import setup_logging
from src.utils.reproducibility import set_seed
from configs.config_loader import ConfigLoader
from configs.constants import (
    AG_NEWS_CLASSES,
    AG_NEWS_NUM_CLASSES,
    DATA_DIR,
    OUTPUT_DIR
)

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
set_seed(42)
logger = setup_logging('preprocessing_tutorial')

# Download NLTK resources
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

print("Text Preprocessing Tutorial")
print("="*50)
print(f"Project Root: {PROJECT_ROOT}")

## 2. Load Configuration

In [ ]:
# Load preprocessing configuration
config_loader = ConfigLoader()

# Load preprocessing config
preprocessing_config = config_loader.load_config('data/preprocessing/standard.yaml')

# Load model config to ensure compatibility
model_config = config_loader.load_config('models/single/deberta_v3_xlarge.yaml')

# Tutorial overrides for demonstration
tutorial_config = {
    'max_samples': 1000,
    'max_length': 256,
    'batch_size': 8,
    'model_name': 'microsoft/deberta-v3-base',
    'use_cache': True
}

print("Preprocessing Configuration:")
print("="*50)
for key, value in tutorial_config.items():
    print(f"  {key}: {value}")

## 3. Data Loading

In [ ]:
# Load AG News dataset
data_config = AGNewsConfig(
    data_dir=DATA_DIR / "processed",
    max_samples=tutorial_config['max_samples'],
    use_cache=tutorial_config['use_cache']
)

print("Loading datasets...")
train_dataset = AGNewsDataset(data_config, split="train")
val_dataset = AGNewsDataset(data_config, split="validation")

print(f"\nDataset loaded:")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Validation samples: {len(val_dataset)}")

# Extract sample texts for demonstration
sample_texts = train_dataset.texts[:10]
sample_labels = train_dataset.labels[:10]

# Display first sample
print(f"\nFirst sample:")
print(f"  Label: {sample_labels[0]} ({train_dataset.label_names[0]})")
print(f"  Text: {sample_texts[0][:200]}...")

## 4. Text Cleaning Strategies

In [ ]:
# Initialize text cleaners
from src.data.preprocessing.text_cleaner import get_minimal_cleaner, get_aggressive_cleaner

# Minimal cleaning (recommended for transformers)
minimal_cleaner = get_minimal_cleaner()

# Standard cleaning
standard_config = CleaningConfig(
    lowercase=False,  # Preserve casing for transformers
    remove_punctuation=False,
    remove_digits=False,
    remove_urls=True,
    remove_emails=True,
    remove_html_tags=True,
    normalize_whitespace=True
)
standard_cleaner = TextCleaner(standard_config)

# Aggressive cleaning (for comparison)
aggressive_cleaner = get_aggressive_cleaner()

print("Text Cleaning Comparison:")
print("="*50)

test_text = "Check out this AMAZING deal at https://example.com! <b>Only $99.99</b> #BestDeal @user"

print(f"Original text:")
print(f"  {test_text}")

minimal_cleaned = minimal_cleaner.clean(test_text)
print(f"\nMinimal cleaning:")
print(f"  {minimal_cleaned}")

standard_cleaned = standard_cleaner.clean(test_text)
print(f"\nStandard cleaning:")
print(f"  {standard_cleaned}")

aggressive_cleaned = aggressive_cleaner.clean(test_text)
print(f"\nAggressive cleaning:")
print(f"  {aggressive_cleaned}")

# Apply to AG News sample
print(f"\nAG News sample (standard cleaning):")
sample_cleaned = standard_cleaner.clean(sample_texts[0])
print(f"  Before: {sample_texts[0][:150]}...")
print(f"  After:  {sample_cleaned[:150]}...")

## 5. Tokenization for Transformer Models

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(tutorial_config['model_name'])

print(f"Tokenizer Configuration:")
print("="*50)
print(f"Model: {tutorial_config['model_name']}")
print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Max length: {tutorial_config['max_length']}")
print(f"Special tokens: {list(tokenizer.special_tokens_map.keys())}")

# Tokenize sample text
sample_text = sample_cleaned
encoding = tokenizer(
    sample_text,
    truncation=True,
    max_length=tutorial_config['max_length'],
    padding='max_length',
    return_tensors='pt'
)

print(f"\nTokenization Example:")
print(f"  Original text length: {len(sample_text)} chars")
print(f"  Input IDs shape: {encoding['input_ids'].shape}")
print(f"  Attention mask shape: {encoding['attention_mask'].shape}")
print(f"  Number of tokens (non-padding): {encoding['attention_mask'].sum().item()}")

# Show token details
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0][:20])
print(f"\nFirst 20 tokens:")
print(f"  {tokens}")

# Decode to verify
decoded = tokenizer.decode(encoding['input_ids'][0], skip_special_tokens=True)
print(f"\nDecoded text:")
print(f"  {decoded[:150]}...")

## 6. Batch Preprocessing Pipeline

In [ ]:
class PreprocessingPipeline:
    """
    Complete preprocessing pipeline for AG News.
    
    Following pipeline design patterns from:
        Pedregosa et al. (2011): "Scikit-learn: Machine Learning in Python"
    """
    
    def __init__(self,
                 tokenizer: AutoTokenizer,
                 text_cleaner: Optional[TextCleaner] = None,
                 max_length: int = 256):
        self.tokenizer = tokenizer
        self.text_cleaner = text_cleaner or get_minimal_cleaner()
        self.max_length = max_length
    
    def process_batch(self,
                     texts: List[str],
                     labels: Optional[List[int]] = None) -> Dict[str, torch.Tensor]:
        """
        Process batch of texts through complete pipeline.
        """
        # Step 1: Clean texts
        cleaned_texts = [self.text_cleaner.clean(text) for text in texts]
        
        # Step 2: Tokenize
        encoding = self.tokenizer(
            cleaned_texts,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        
        # Step 3: Add labels if provided
        if labels is not None:
            encoding['labels'] = torch.tensor(labels, dtype=torch.long)
        
        return encoding
    
    def process_dataset(self,
                       dataset: AGNewsDataset,
                       batch_size: int = 32,
                       show_progress: bool = True) -> Dict[str, torch.Tensor]:
        """
        Process entire dataset in batches.
        """
        all_input_ids = []
        all_attention_masks = []
        all_labels = []
        
        num_batches = (len(dataset) + batch_size - 1) // batch_size
        
        iterator = range(0, len(dataset), batch_size)
        if show_progress:
            iterator = tqdm(iterator, desc="Processing", total=num_batches)
        
        for i in iterator:
            batch_texts = dataset.texts[i:i+batch_size]
            batch_labels = dataset.labels[i:i+batch_size]
            
            batch_encoding = self.process_batch(batch_texts, batch_labels)
            
            all_input_ids.append(batch_encoding['input_ids'])
            all_attention_masks.append(batch_encoding['attention_mask'])
            all_labels.append(batch_encoding['labels'])
        
        return {
            'input_ids': torch.cat(all_input_ids, dim=0),
            'attention_mask': torch.cat(all_attention_masks, dim=0),
            'labels': torch.cat(all_labels, dim=0)
        }

# Create and test pipeline
pipeline = PreprocessingPipeline(
    tokenizer=tokenizer,
    text_cleaner=standard_cleaner,
    max_length=tutorial_config['max_length']
)

# Process a small batch
batch_texts = train_dataset.texts[:tutorial_config['batch_size']]
batch_labels = train_dataset.labels[:tutorial_config['batch_size']]

batch_result = pipeline.process_batch(batch_texts, batch_labels)

print("Batch Processing Results:")
print("="*50)
print(f"Batch size: {tutorial_config['batch_size']}")
print(f"Input IDs shape: {batch_result['input_ids'].shape}")
print(f"Attention mask shape: {batch_result['attention_mask'].shape}")
print(f"Labels shape: {batch_result['labels'].shape}")

## 7. Feature Extraction

In [ ]:
# Initialize feature extractor
feature_config = FeatureExtractionConfig(
    extract_length_features=True,
    extract_readability_features=True,
    extract_pos_features=False,  # Skip for speed
    extract_entity_features=False
)

feature_extractor = FeatureExtractor(feature_config)

# Extract features from samples
print("Feature Extraction:")
print("="*50)

features_list = []
for i, text in enumerate(sample_texts[:5]):
    features = feature_extractor.extract(text)
    features_list.append(features)
    
    if i == 0:  # Show first example
        print(f"\nSample {i+1} features:")
        for key, value in list(features.items())[:10]:
            if isinstance(value, float):
                print(f"  {key:20}: {value:.3f}")
            else:
                print(f"  {key:20}: {value}")

# Convert to DataFrame for analysis
features_df = pd.DataFrame(features_list)

print("\nFeature Statistics:")
print(features_df[['word_count', 'char_count', 'avg_word_length']].describe().round(2))

# Correlation with labels
features_df['label'] = sample_labels[:5]
correlation = features_df.corr()['label'].sort_values(ascending=False)

print("\nFeature-Label Correlation (top 5):")
for feature, corr in correlation.head(6).items():
    if feature != 'label':
        print(f"  {feature:20}: {corr:.3f}")

## 8. DataLoader Integration

In [ ]:
from torch.utils.data import DataLoader, Dataset

class PreprocessedDataset(Dataset):
    """
    Dataset with integrated preprocessing.
    
    Following PyTorch dataset best practices from:
        Paszke et al. (2019): "PyTorch: An Imperative Style, High-Performance Deep Learning Library"
    """
    
    def __init__(self,
                 texts: List[str],
                 labels: List[int],
                 pipeline: PreprocessingPipeline):
        self.texts = texts
        self.labels = labels
        self.pipeline = pipeline
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Process single sample
        result = self.pipeline.process_batch(
            [self.texts[idx]],
            [self.labels[idx]]
        )
        
        # Remove batch dimension
        return {
            'input_ids': result['input_ids'].squeeze(0),
            'attention_mask': result['attention_mask'].squeeze(0),
            'labels': result['labels'].squeeze(0)
        }

# Create preprocessed dataset
preprocessed_dataset = PreprocessedDataset(
    texts=train_dataset.texts[:100],
    labels=train_dataset.labels[:100],
    pipeline=pipeline
)

# Create DataLoader
from src.data.loaders.dataloader import create_dataloaders

dataloader = DataLoader(
    preprocessed_dataset,
    batch_size=tutorial_config['batch_size'],
    shuffle=True,
    num_workers=2,
    pin_memory=torch.cuda.is_available()
)

# Test DataLoader
batch = next(iter(dataloader))

print("DataLoader Integration Test:")
print("="*50)
print(f"Dataset size: {len(preprocessed_dataset)}")
print(f"Number of batches: {len(dataloader)}")
print(f"\nBatch shapes:")
for key, value in batch.items():
    print(f"  {key}: {value.shape}")

## 9. Performance Optimization

In [ ]:
import time

def benchmark_preprocessing(
    texts: List[str],
    pipeline: PreprocessingPipeline,
    batch_sizes: List[int] = [1, 8, 16, 32]
) -> pd.DataFrame:
    """
    Benchmark preprocessing performance.
    
    Following benchmarking practices from:
        PyTorch Performance Tuning Guide
    """
    results = []
    
    for batch_size in batch_sizes:
        # Sequential processing
        start = time.time()
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            _ = pipeline.process_batch(batch)
        seq_time = time.time() - start
        
        # Batch processing
        start = time.time()
        _ = pipeline.process_batch(texts)
        batch_time = time.time() - start
        
        results.append({
            'batch_size': batch_size,
            'sequential_time': seq_time,
            'batch_time': batch_time,
            'speedup': seq_time / batch_time,
            'throughput': len(texts) / batch_time
        })
    
    return pd.DataFrame(results)

# Benchmark with different configurations
test_texts = train_dataset.texts[:100]
benchmark_results = benchmark_preprocessing(test_texts, pipeline)

print("Preprocessing Performance Benchmark:")
print("="*50)
print(benchmark_results.to_string(index=False))

# Memory usage analysis
import sys

# Process dataset
processed_data = pipeline.process_dataset(
    train_dataset,
    batch_size=32,
    show_progress=False
)

# Calculate memory usage
memory_usage = {
    'input_ids': processed_data['input_ids'].element_size() * processed_data['input_ids'].nelement() / 1024**2,
    'attention_mask': processed_data['attention_mask'].element_size() * processed_data['attention_mask'].nelement() / 1024**2,
    'labels': processed_data['labels'].element_size() * processed_data['labels'].nelement() / 1024**2
}

print("\nMemory Usage:")
for key, value in memory_usage.items():
    print(f"  {key}: {value:.2f} MB")
print(f"  Total: {sum(memory_usage.values()):.2f} MB")

## 10. Save Preprocessed Data

In [ ]:
def save_preprocessed_dataset(
    dataset: AGNewsDataset,
    pipeline: PreprocessingPipeline,
    output_path: Path,
    batch_size: int = 32
) -> Path:
    """
    Save preprocessed dataset for future use.
    
    Following data serialization best practices from:
        PyTorch Documentation: "Saving and Loading Models"
    """
    ensure_dir(output_path.parent)
    
    print(f"Preprocessing {len(dataset)} samples...")
    
    # Process entire dataset
    processed_data = pipeline.process_dataset(
        dataset,
        batch_size=batch_size,
        show_progress=True
    )
    
    # Add metadata
    processed_data['metadata'] = {
        'num_samples': len(dataset),
        'max_length': pipeline.max_length,
        'tokenizer': tutorial_config['model_name'],
        'preprocessing_date': pd.Timestamp.now().isoformat(),
        'dataset_split': dataset.split
    }
    
    # Save to disk
    torch.save(processed_data, output_path)
    
    file_size = output_path.stat().st_size / 1024**2
    print(f"\nSaved to: {output_path}")
    print(f"File size: {file_size:.2f} MB")
    
    return output_path

# Save preprocessed training data
output_dir = OUTPUT_DIR / "tutorial" / "preprocessed"
train_output = output_dir / "train_preprocessed.pt"

saved_path = save_preprocessed_dataset(
    dataset=train_dataset,
    pipeline=pipeline,
    output_path=train_output,
    batch_size=32
)

# Verify saved data
print("\nVerifying saved data...")
loaded_data = torch.load(saved_path)

print("Loaded data structure:")
for key in loaded_data.keys():
    if key != 'metadata':
        print(f"  {key}: shape {loaded_data[key].shape}, dtype {loaded_data[key].dtype}")
    else:
        print(f"  {key}: {loaded_data[key]}")

## 11. Conclusions and Next Steps

### Preprocessing Summary

This tutorial demonstrated fundamental text preprocessing concepts:

1. **Environment Setup**: Configured preprocessing environment with necessary libraries
2. **Data Loading**: Loaded AG News dataset with configurable parameters
3. **Text Cleaning**: Implemented minimal, standard, and aggressive cleaning strategies
4. **Tokenization**: Applied DeBERTa-v3 tokenizer for transformer compatibility
5. **Pipeline Design**: Created modular preprocessing pipeline
6. **Feature Extraction**: Extracted linguistic features for analysis
7. **DataLoader Integration**: Integrated preprocessing with PyTorch DataLoader
8. **Performance Optimization**: Benchmarked and optimized preprocessing speed
9. **Data Persistence**: Saved preprocessed data for reproducibility

### Key Takeaways

1. **Tokenizer Consistency**: Always use the same tokenizer for preprocessing and model training
2. **Minimal Cleaning**: Transformer models benefit from preserving original text structure
3. **Batch Processing**: Batch tokenization provides 3-5x speedup over sequential processing
4. **Memory Efficiency**: Preprocessed data requires ~3-4x more memory than raw text
5. **Pipeline Modularity**: Modular design enables easy experimentation with different strategies

### Next Steps

1. **Advanced Preprocessing**:
   - Implement data augmentation (back-translation, paraphrasing)
   - Apply domain-specific cleaning rules
   - Use subword regularization techniques

2. **Optimization**:
   - Profile preprocessing bottlenecks with cProfile
   - Implement multi-processing for large datasets
   - Use memory mapping for out-of-core processing

3. **Integration**:
   - Connect with model training pipeline
   - Implement streaming preprocessing for real-time inference
   - Create preprocessing microservice

4. **Production**:
   - Build scalable preprocessing with Apache Beam/Spark
   - Implement preprocessing cache with Redis
   - Monitor preprocessing latency and throughput

### References

For deeper understanding, consult:
- Preprocessing documentation: `docs/user_guide/data_preparation.md`
- Model training: `notebooks/tutorials/03_model_training_basics.ipynb`
- Advanced techniques: `notebooks/tutorials/05_prompt_engineering.ipynb`
- API integration: `notebooks/tutorials/07_api_usage.ipynb`